In [1]:
# Copyright 2015 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Trains and Evaluates the MNIST network using a feed dictionary."""
# pylint: disable=missing-docstring
import sys
sys.path.append('../../')
import os
import time
import numpy
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import input_test
import math
import numpy as np
from i3d import InceptionI3d
from utils import *

# Basic model parameters as external flags.
flags = tf.app.flags
gpu_num = 1
flags.DEFINE_integer('batch_size', 1, 'Batch size.')
flags.DEFINE_integer('num_frame_per_clib', 64, 'Nummber of frames per clib')
flags.DEFINE_integer('crop_size', 224, 'Crop_size')
flags.DEFINE_integer('rgb_channels', 3, 'Channels for input')
flags.DEFINE_integer('classics', 51, 'The num of class')
FLAGS = flags.FLAGS

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
outfile = './rgb_scratch_data_split1/train64/'


def run_training():
    # Get the sets of images and labels for training, validation, and
    # Tell TensorFlow that the model will be built into the default Graph.
    pre_model_save_dir = "./models/HMDB_scratch_fix/rgb_scratch_10000_6_64_0.0001_decay_low_split1"
    test_list_file = '../../list/hmdb_list/trainlist1.list'
    file = list(open(test_list_file, 'r'))
    num_test_videos = len(file)
    print("Number of test videos={}".format(num_test_videos))
    with tf.Graph().as_default():
        rgb_images_placeholder, _, labels_placeholder, is_training = placeholder_inputs(
                        FLAGS.batch_size * gpu_num,
                        FLAGS.num_frame_per_clib,
                        FLAGS.crop_size,
                        FLAGS.rgb_channels
                        )

        with tf.variable_scope('RGB'):
            logit, _ = InceptionI3d(
                                num_classes=FLAGS.classics,
                                spatial_squeeze=True,
                                final_endpoint='Mixed_5c',
                                name='inception_i3d'
                                )(rgb_images_placeholder, is_training)
        norm_score = tf.nn.softmax(logit)

        # Create a saver for writing training checkpoints.
        saver = tf.train.Saver()
        init = tf.global_variables_initializer()

        # Create a session for running Ops on the Graph.
        sess = tf.Session(
                        config=tf.ConfigProto(allow_soft_placement=True)
                        )
        sess.run(init)

    ckpt = tf.train.get_checkpoint_state(pre_model_save_dir)
    if ckpt and ckpt.model_checkpoint_path:
        print ("loading checkpoint %s,waiting......" % ckpt.model_checkpoint_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
        print ("load complete!")

    all_steps = num_test_videos
    top1_list = []
    for step in xrange(all_steps):
        start_time = time.time()
        s_index = 0
        predicts = []
        data = {}
        top1 = False
        data_new = np.array([])
        while True:
            val_images, _, val_labels, s_index, is_end = input_test.read_clip_and_label(
                            filename=file[step],
                            batch_size=FLAGS.batch_size * gpu_num,
                            s_index=s_index,
                            num_frames_per_clip=FLAGS.num_frame_per_clib,
                            crop_size=FLAGS.crop_size,
                            )

            data_features = sess.run(logit,
                               feed_dict={
                                            rgb_images_placeholder: val_images,
                                            labels_placeholder: val_labels,
                                            is_training: False
                                            })

            temp = np.mean(data_features, axis=2)
            tmp = np.mean(temp, axis=2)
            tmp = np.squeeze(tmp)

            if data_new.shape[0] == 0:
                data_new = tmp
            else:
                data_new = np.concatenate([data_new,tmp])

            labell = np.zeros((1,51))
            labell[0,int(val_labels)] = 1

            #print(labell)

            if is_end:
                ndata = int(data_new.shape[0])
                nn = 64
                if ndata <=nn:
                    data_new = np.concatenate([data_new,data_new],  axis=0)
                    data_new = np.concatenate([data_new,data_new],  axis=0)
                    ndata = int(data_new.shape[0])
                
                sampling = ndata/nn
                print(sampling)
                print(data_new.shape)
                ndata2 = data_new[0:nn*sampling,:]
                ndata3 = ndata2[0::sampling,:]

                data['feat'] = ndata3
                data['label'] = labell
                
                temp = file[step].split("/")
                namefile = temp[6].split(" ")
                print(namefile[0])

                datasave = outfile+namefile[0]
                print(datasave)
                np.save(datasave, data)
                break


def main(_):
    run_training()


if __name__ == '__main__':
    tf.app.run()


/usr/local/lib/python2.7/dist-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)


Number of test videos=3570
loading checkpoint ./models/HMDB_scratch_fix/rgb_scratch_10000_6_64_0.0001_decay_low_split1/i3d_hmdb_model_split1-14999,waiting......
INFO:tensorflow:Restoring parameters from ./models/HMDB_scratch_fix/rgb_scratch_10000_6_64_0.0001_decay_low_split1/i3d_hmdb_model_split1-14999
load complete!
Loading a video clip from /home/ee401_2/Documents/HMDB_new/cartwheel/Geena_und_die_Radschl_ge___;-_cartwheel_f_cm_np1_fr_med_2...strat_frame = 0
Loading a video clip from /home/ee401_2/Documents/HMDB_new/cartwheel/Geena_und_die_Radschl_ge___;-_cartwheel_f_cm_np1_fr_med_2...strat_frame = 64
Loading a video clip from /home/ee401_2/Documents/HMDB_new/cartwheel/Geena_und_die_Radschl_ge___;-_cartwheel_f_cm_np1_fr_med_2...strat_frame = 128
Loading a video clip from /home/ee401_2/Documents/HMDB_new/cartwheel/Geena_und_die_Radschl_ge___;-_cartwheel_f_cm_np1_fr_med_2...strat_frame = 192
2
(128, 1024)
Geena_und_die_Radschl_ge___;-_cartwheel_f_cm_np1_fr_med_2
./rgb_scratch_data_split

Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np2_le_med_2...strat_frame = 128
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np2_le_med_2...strat_frame = 192
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np2_le_med_2...strat_frame = 256
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np2_le_med_2...strat_frame = 320
3
(192, 1024)
Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np2_le_med_2
./rgb_scratch_data_split1/train64/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np2_le_med_2
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Blonde_being_brushed_brush_hair_f_nm_np2_ri_med_0...strat_frame = 0
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Blonde_being

Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Brunette_Foxyanya_ultra_silky_long_hair_brushing_hairjob_brush_hair_h_cm_np2_ri_goo_6...strat_frame = 128
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Brunette_Foxyanya_ultra_silky_long_hair_brushing_hairjob_brush_hair_h_cm_np2_ri_goo_6...strat_frame = 192
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Brunette_Foxyanya_ultra_silky_long_hair_brushing_hairjob_brush_hair_h_cm_np2_ri_goo_6...strat_frame = 256
2
(160, 1024)
Brunette_Foxyanya_ultra_silky_long_hair_brushing_hairjob_brush_hair_h_cm_np2_ri_goo_6
./rgb_scratch_data_split1/train64/Brunette_Foxyanya_ultra_silky_long_hair_brushing_hairjob_brush_hair_h_cm_np2_ri_goo_6
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Brunette_Foxyanya_ultra_silky_long_hair_brushing_hairjob_brush_hair_h_nm_np2_ri_goo_5...strat_frame = 0
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Brunette

Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Brushing_my_Long_Hair__February_2009_brush_hair_u_nm_np1_ba_goo_2...strat_frame = 256
2
(160, 1024)
Brushing_my_Long_Hair__February_2009_brush_hair_u_nm_np1_ba_goo_2
./rgb_scratch_data_split1/train64/Brushing_my_Long_Hair__February_2009_brush_hair_u_nm_np1_ba_goo_2
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Brushing_my_hair_-_December_2008_brush_hair_u_cm_np1_ba_goo_1...strat_frame = 0
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Brushing_my_hair_-_December_2008_brush_hair_u_cm_np1_ba_goo_1...strat_frame = 64
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Brushing_my_hair_-_December_2008_brush_hair_u_cm_np1_ba_goo_1...strat_frame = 128
Loading a video clip from /home/ee401_2/Documents/HMDB_new/brush_hair/Brushing_my_hair_-_December_2008_brush_hair_u_cm_np1_ba_goo_1...strat_frame = 192
Loading a video clip from /home/ee401_2/Documents/HMDB_new/

...

2
(128, 1024)
Newall_Green_High_Students_Waving_Goodbye_wave_u_cm_np1_fr_med_0
./rgb_scratch_data_split1/train64/Newall_Green_High_Students_Waving_Goodbye_wave_u_cm_np1_fr_med_0
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/Newall_Green_High_Students_Waving_Goodbye_wave_u_cm_np3_fr_med_1...strat_frame = 0
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/Newall_Green_High_Students_Waving_Goodbye_wave_u_cm_np3_fr_med_1...strat_frame = 64
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/Newall_Green_High_Students_Waving_Goodbye_wave_u_cm_np3_fr_med_1...strat_frame = 128
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/Newall_Green_High_Students_Waving_Goodbye_wave_u_cm_np3_fr_med_1...strat_frame = 192
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/Newall_Green_High_Students_Waving_Goodbye_wave_u_cm_np3_fr_med_1...strat_frame = 256
2
(160, 1024)
Newall_Green_High_Students_Waving_Goodbye_wave_u_cm_np3_fr_med_1
.

Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/Song_I_Can_Wave_My_Hands_-_Cullen_s_Abc_s_wave_u_cm_np1_fr_med_2...strat_frame = 128
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/Song_I_Can_Wave_My_Hands_-_Cullen_s_Abc_s_wave_u_cm_np1_fr_med_2...strat_frame = 192
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/Song_I_Can_Wave_My_Hands_-_Cullen_s_Abc_s_wave_u_cm_np1_fr_med_2...strat_frame = 256
2
(160, 1024)
Song_I_Can_Wave_My_Hands_-_Cullen_s_Abc_s_wave_u_cm_np1_fr_med_2
./rgb_scratch_data_split1/train64/Song_I_Can_Wave_My_Hands_-_Cullen_s_Abc_s_wave_u_cm_np1_fr_med_2
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/UNREPEATABLE_wave_f_nm_np1_fr_med_5...strat_frame = 0
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/UNREPEATABLE_wave_f_nm_np1_fr_med_5...strat_frame = 64
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/UNREPEATABLE_wave_f_nm_np1_fr_med_5...strat_frame = 128
Loading a video cl

Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/prelinger_LetsBeGo1953_wave_u_cm_np10_ba_med_3...strat_frame = 64
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/prelinger_LetsBeGo1953_wave_u_cm_np10_ba_med_3...strat_frame = 128
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/prelinger_LetsBeGo1953_wave_u_cm_np10_ba_med_3...strat_frame = 192
2
(128, 1024)
prelinger_LetsBeGo1953_wave_u_cm_np10_ba_med_3
./rgb_scratch_data_split1/train64/prelinger_LetsBeGo1953_wave_u_cm_np10_ba_med_3
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/prideandprejudice1_wave_f_nm_np1_ri_med_14...strat_frame = 0
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/prideandprejudice1_wave_f_nm_np1_ri_med_14...strat_frame = 64
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/prideandprejudice1_wave_f_nm_np1_ri_med_14...strat_frame = 128
Loading a video clip from /home/ee401_2/Documents/HMDB_new/wave/prideandprejudice1_wave_

SystemExit: 

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2886: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
